In [1]:
# This is to try to analyze the thousands of determining equations

# Learning Pandas right now, so I'm going to try to use Pandas

import numpy as np
import pandas as pd

In [2]:
raw_eqs = pd.read_csv('RMHD_CHM_DE.csv', header=None)

In [3]:
pd.set_option('display.max_colwidth', None)
raw_eqs

,0
0,"-Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
1,"Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
2,"-Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
3,"Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
4,"-3*Derivative[0, 0, 0, 0, 0, 0, 0, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
...,...
3850,"2*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - 3*Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
3851,"2*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
3852,"-2*Derivative[0, 0, 1, 0, 0, 0, 1, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8] - 4*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] + 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + 3*Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*Derivative[1, 1, 0, 0, 0, 0, 0, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8] + α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] + α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] == 0"
3853,"-3*Derivative[0, 0, 1, 0, 0, 0, 1, 0][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*η*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + Derivative[1, 0, 0, 0, 0, 0, 1, 0][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*Derivative[1, 1, 0, 0, 0, 0, 0, 0][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - η*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - η*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] == 0"


##### Initial Thoughts

Going to need to figure out how to modify dataframes. Need to go through another lesson or so on codecademy.
I want to strip ' == 0' and replace 'Derivative' with something else. This should be simple with string methods.
I also think I need to figure out a way to analyze the specific derivative. Maybe someway to interpret the 'list of integers' as a list of ints?
I could just split it along the commas, the length of it is always constant.

I think I can probably split on the +/-s, and also on the * to separate into coefficients.
As of now, the data is a little dirty because Excel can't handle Greek characters.
I can either do a replacement in the CSV, Python, or just rerun the mathematica nb with other characters.

I think the first step is to get them into "standard forms," and then get rid of duplicates. 
This should reduce the number of equations significantly. 

In [4]:
# A function that will interpret the string as a list:

def intlist(s):
    s = s.strip('[')
    s = s.strip(']')
    l = s.split(', ')
    interpreted = []
    for n in l:
        interpreted.append(int(n))
    return interpreted

# Now we can interpret the list after the 'Derivative' as a list of integers!

# Test:
# print(intlist('[1, 0, 1, 3, 2, 1]'))
# print(type(intlist('[1, 0, 1, 3, 2, 1]')[0]))

# This seems to work great. Cantwell's package always produces strings like this after derivatives.

In [5]:
# print(raw_eqs.iloc[36])
# print(raw_eqs.iloc[251:256])

# Somehow Python seems to be able to read the greek characters, so that's good
# I am noticing that the first line of the CSV is the first equation, but pandas is interpreting that as a column label.
# Not sure how to fix that easily. Might be better to just import the CSV in a different way.
# Figured it out! It's just header=None in the .read_csv()

##### More thoughts

All of the equations are a linear combination of derivatives of the xses/etas = 0, so it shouldn't be too hard to categorize them.
The maximum derivative is 3, and we have 5 independent variables, and 3 dependent variables
Except there are a maximum of 3 total derivatives,so that reduces the total variations of derivatives, and all of these derivatives are acting on a xse or an eta.

I'm thinking I can probably define a class that can hopefully just interpret an equation string in its initialization and then store it in some standard form, that doesn't care about order. Not sure exactly how to do that yet. Maybe some kind of ndarray?

Maybe instead of an object for each, I can use a DataFrame somehow? Each column representing a derivative with respect to xse 1,2,3,4,5 and eta 1,2,3 and then each column has a list that I get from the intlist function I made. And then maybe one of the columns are the coefficients, stored as a list of strings, since we have the greek constants α and η? Maybe one extra column to show if there are any of the constants being multiplied into the coefficients? Not quite sure what to do after that.

I would like to try to make this work for any differential equation with any number of independent and dependent variables if possible. I think this is probably doable with DataFrames. The problem is that there can be multiple derivatives of the same xse or eta in the same equation, and I need to work out a way to account for that.

In [6]:
raw_eqs[0] = raw_eqs[0].apply(lambda eq: eq.strip(' == 0'))

# Truncating the repetitive ending off the equations

In [7]:
raw_eqs

,0
0,"-Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1][z1, z2, z3, z4, z5, z6, z7, z8]"
1,"Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1][z1, z2, z3, z4, z5, z6, z7, z8]"
2,"-Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8]"
3,"Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8]"
4,"-3*Derivative[0, 0, 0, 0, 0, 0, 0, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8]"
...,...
3850,"2*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - 3*Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]"
3851,"2*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]"
3852,"-2*Derivative[0, 0, 1, 0, 0, 0, 1, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8] - 4*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] + 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + 3*Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*Derivative[1, 1, 0, 0, 0, 0, 0, 1][xse1][z1, z2, z3, z4, z5, z6, z7, z8] + α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] + α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]"
3853,"-3*Derivative[0, 0, 1, 0, 0, 0, 1, 0][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*η*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] + Derivative[1, 0, 0, 0, 0, 0, 1, 0][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*Derivative[1, 1, 0, 0, 0, 0, 0, 0][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - η*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - η*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]"


In [8]:
# Want to write a function that will split the equations into terms. The terms are always separated by a + or -.
# One issue is that some equations begin with a negative. I think we want to store the sign of the term and the
# term itself separately. We can incorporate the sign into the coefficient later.

def get_terms(equation):
    equation = equation.replace('+','-')
    # Break the equation into it's terms
    term_list = equation.split('-')
    # Now strip the whitespace
    terms = [term.strip(' ') for term in term_list]
    if terms[0] == '':
        terms.pop(0)
    # This gets rid of the empty string that results from the equation starting with a negative sign.
    return terms

# test_eq = raw_eqs[0].iloc[139]
# print(test_eq)
# get_terms(test_eq)

# This function seems to work, but I will also need to make a function that gets the signs

In [9]:
# Now I want to write a function to get the signs

def get_signs(equation):
    sign_array = []
    if equation[0] != "-":
        sign_array.append(1)
    for char in equation:
        if char == "+":
            sign_array.append(1)
        elif char == '-':
            sign_array.append(-1)
        else:
            continue
    return sign_array


test_eq = raw_eqs[0].iloc[818]
print(test_eq)
get_signs(test_eq)

# This function works. Not sure if the variable length dependent on how many terms there are will matter.

Derivative[0, 0, 0, 0, 0, 0, 1, 2][eta1][z1, z2, z3, z4, z5, z6, z7, z8] + 3*Derivative[0, 0, 0, 0, 0, 1, 0, 2][eta2][z1, z2, z3, z4, z5, z6, z7, z8] + Derivative[0, 0, 1, 0, 0, 0, 0, 2][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 0, 1, 0, 0, 1, 0, 1][xse5][z1, z2, z3, z4, z5, z6, z7, z8]


[1, 1, 1, -1]

In [10]:
raw_eqs['Terms'] = raw_eqs[0].apply(get_terms)
raw_eqs['Signs'] = raw_eqs[0].apply(get_signs)

# print(raw_eqs)
# Now we have the terms and signs separated out. The terms are all in a fairly uniform format:
# coefficient*Derivative[derivative list][function][z1, z2, z3, z4, z5, z6, z7, z8]

In [15]:
# Want to write a 'process_term' function that will read the functions as coefficient, derivative list and the function.

def process_term(term):
    # First need to separate coefficients
    parts = term.split('*')
    # Make the numerical coefficient into an integer
    if len(parts) > 1:
        try:
            parts[0] = int(parts[0])
        except:
        # Here we catch any greek letters, and then put an integer of 1 at the beginning of the list
            parts = [1] + parts
            
    # Might be a potential issue if two greek letters are multiplied together, but I don't think that happens, at least
    # in this set of data.
    # Need to strip in 2 stages as to not delete the function information
    parts[-1] = parts[-1].strip('z1, z2, z3, z4, z5, z6, z7, z8]').strip('[')
    return parts

# Going to define a more convinient form for analyzing the term lists
def process_terms(terms):
    return [process_term(term) for term in terms]
    

# print(process_term('η*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]'))
# This seems to work how I want it to.

In [16]:
raw_eqs['Terms_1'] = raw_eqs['Terms'].apply(process_terms)

# raw_eqs.Terms_1

0                                                                                                                                                                                                                                                                                                                                                         [[Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1]]]
1                                                                                                                                                                                                                                                                                                                                                         [[Derivative[0, 0, 0, 0, 0, 0, 0, 1][eta1]]]
2                                                                                                                                                                                                                         

In [27]:
# Function to process the derivatives. All derivatives have a format of Derivative intlist function.

def process_derivative(derivative):
    derivative = derivative.strip('Derivative')
    int_list = derivative[:derivative.find(']')+1]
    fnc = derivative[derivative.find(']')+1:].strip('[]')
    return fnc, intlist(int_list)
    

print(process_derivative('Derivative[0, 0, 1, 0, 0, 0, 1, 1][xse1]'))

('xse1', [0, 0, 1, 0, 0, 0, 1, 1])
